In [ ]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
import easy_excel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
import sklearn.ensemble
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB  
import subprocess
from sklearn.utils import shuffle
import itertools
from sklearn.ensemble import GradientBoostingClassifier
import sys


In [ ]:
def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall=0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

In [12]:
import pandas as pd
import numpy as ny
import sys
from sklearn.preprocessing import MinMaxScaler
import numpy as np
path1="data/human_PCP.csv"
path2="data/human_AthMethPre_2RFH.csv"
path3="data/human_AthMethPre_2RFH_PCP.csv"





First_One=pd.read_csv(path1,header=None,index_col=None)#sys.argv[1])
end=len(First_One.values[0])
First_One=First_One.values[:,0:end-1]
First_One=pd.DataFrame(First_One).astype(float)
#PseDNC_=pd.read_csv(sys.argv[2],header=None,index_col=None)
Second_One=pd.read_csv(path2,header=None,index_col=None)
Second_One=pd.DataFrame(Second_One).astype(float)
print "first_file_num:",len(First_One)
print "first_file_length:",len(First_One.values[0])
print "second_file_num:",len(Second_One)
print "second_file_length:",len(Second_One.values[0])
Third_One=pd.concat([First_One,Second_One],axis=1)
print "output_file_num:",len(Third_One)
print "output_file_len:",len(Third_One.values[0])

scaler=MinMaxScaler()
Third_One=scaler.fit_transform(np.array(Third_One))
print "normalization"

# scaler.fit_transform(np.array(RFH_PseDNC))
pd.DataFrame(Third_One).to_csv(path3,header=None,index=False)
# print RFH_PseDNC

first_file_num: 2260
first_file_length: 399
second_file_num: 2260
second_file_length: 699
output_file_num: 2260
output_file_len: 1098
normalization


In [29]:
"""
    cross validation
"""
datapath =path3
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) / 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) / 2)))
Y.extend(Y2)
Y = np.array(Y)
svc = svm.SVC()
parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
clf = GridSearchCV(svc, parameters, cv=5, n_jobs=12, scoring='accuracy')
print X.shape
clf.fit(X, Y)

C=clf.best_params_['C']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_
gamma=clf.best_params_['gamma']
y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=2260,n_jobs=12)
ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=12).mean()
y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=12,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
# pd.DataFrame(predict_save).to_csv(path+outputname+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
# easy_excel.save("svm_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+name+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

(2260, 1098)


ValueError: All the n_groups for individual classes are less than n_splits=2260.

In [14]:
"""
    cross validation XGBoost
"""
classifier="xgboost"
datapath =path3
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
parameters = [{'n_estimators':range(1,10,1),
                      'max_depth':range(2,10),
                      'learning_rate':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
                      'subsample':[0.75,0.8,0.85,0.9]
                      }]
clf = GridSearchCV(XGBClassifier(), parameters, cv=10, n_jobs=1, scoring='accuracy')
clf.fit(X, Y)
n_estimators=clf.best_params_['n_estimators']
max_depth=clf.best_params_['max_depth']
learning_rate=clf.best_params_['learning_rate']
subsample=clf.best_params_['subsample']
# joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
# print clf.best_score_
y_predict=cross_val_predict(XGBClassifier(n_estimators=n_estimators,learning_rate=learning_rate,
                                                       subsample=subsample,max_depth=max_depth),X,Y,cv=10,n_jobs=1)
ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
y_predict_prob=cross_val_predict(XGBClassifier(n_estimators=n_estimators,learning_rate=learning_rate,
                                                       subsample=subsample,max_depth=max_depth),X,Y,cv=10,n_jobs=1,method='predict_proba')
predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
predict_save=np.array(predict_save).T
pd.DataFrame(predict_save).to_csv(path+outputname+"_"+classifier+'_predict.csv',header=None,index=False)
ACC=metrics.accuracy_score(Y,y_predict)
precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
F1_Score=metrics.f1_score(Y, y_predict)
F_measure=F1_Score
MCC=metrics.matthews_corrcoef(Y, y_predict)
pos=TP+FN
neg=FP+TN
savedata=[[[classifier+"n_estimators:"+str(n_estimators)+"max_depth:"+str(max_depth)+"learning_rate:"+str(learning_rate)+"subsample:"+str(subsample),ACC,precision, recall,SN, SP,
            GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
print savedata
print X.shape[1]
easy_excel.save(classifier+"_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+classifier+"_"+outputname+'.xls')
# y_predict_proba=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,method="predict_proba")
# Y=pd.DataFrame(Y)    
# y_predict_proba=pd.DataFrame(y_predict_proba)
# y_predict_proba=pd.concat([Y,y_predict_proba],axis=1)
# pd.DataFrame(y_predict_proba).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_label.csv',header=None,index=False)
# y_predict=pd.DataFrame(y_predict)
# y_predict_=pd.concat([Y,y_predict],axis=1)
# pd.DataFrame(y_predict_).to_csv('/home02/chenhuangrong/RFH_ten_cross_validation_predict.csv',header=None,index=False)

[[['xgboostn_estimators:8max_depth:6learning_rate:0.1subsample:0.9', 0.91106194690265485, 1.0, 0.8221238938053097, 0.8221238938053097, 1.0, 0.9067104795938501, 0.90237979601748419, 0.90237979601748419, 0.83544685650109785, 0.91106194690265485, 929.0, 201.0, 0.0, 1130.0, 1130.0, 1130.0]]]
1098


True

In [15]:
# !/use/bin/env python
# encoding:utf-8
import pandas as pd
import numpy as np
import itertools
from sklearn.model_selection import KFold  
from sklearn import svm
from sklearn.model_selection import train_test_split
import math
import easy_excel
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import *
import sklearn.ensemble
import joblib
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
import sys
from sklearn.model_selection import GridSearchCV
import subprocess
from sklearn.utils import shuffle
import itertools
from sklearn.feature_selection import  f_classif

def performance(labelArr, predictArr):
    #labelArr[i] is actual value,predictArr[i] is predict value
    TP = 0.; TN = 0.; FP = 0.; FN = 0.
    for i in range(len(labelArr)):
        if labelArr[i] == 1 and predictArr[i] == 1:
            TP += 1.
        if labelArr[i] == 1 and predictArr[i] == 0:
            FN += 1.
        if labelArr[i] == 0 and predictArr[i] == 1:
            FP += 1.
        if labelArr[i] == 0 and predictArr[i] == 0:
            TN += 1.
    if (TP + FN)==0:
        SN=0
    else:
        SN = TP/(TP + FN) #Sensitivity = TP/P  and P = TP + FN
    if (FP+TN)==0:
        SP=0
    else:
        SP = TN/(FP + TN) #Specificity = TN/N  and N = TN + FP
    if (TP+FP)==0:
        precision=0
    else:
        precision=TP/(TP+FP)
    if (TP+FN)==0:
        recall==0
    else:
        recall=TP/(TP+FN)
    GM=math.sqrt(recall*SP)
    #MCC = (TP*TN-FP*FN)/math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    return precision,recall,SN,SP,GM,TP,TN,FP,FN

"""
    cross validation
"""
datapath =path3
print "start"
train_data = pd.read_csv(datapath, header=None, index_col=None)
Y = list(map(lambda x: 1, xrange(len(train_data) / 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) / 2)))
Y.extend(Y2)
Y = np.array(Y)
F, pval = f_classif(train_data, Y)
idx = np.argsort(F)
selected_list_=idx[::-1]
bestACC=0
bestC=0
bestgamma=0
for select_num in xrange(1,376):
    train_data2=train_data.values
    selected_list_2=selected_list_[xrange(select_num)]
    X_train=pd.DataFrame(train_data2)
    X_train=X_train.iloc[:,selected_list_2]
    X = np.array(X_train)
    svc = svm.SVC()
    parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
    clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
    clf.fit(X, Y)
    C=clf.best_params_['C']
    # joblib.dump(clf,'/home02/chenhuangrong/'+name+'.model')
    gamma=clf.best_params_['gamma']
    y_predict=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=8)
    ROC_AUC_area=metrics.roc_auc_score(Y, y_predict)
    y_predict_prob=cross_val_predict(svm.SVC(kernel='rbf',C=C,gamma=gamma,probability=True),X,Y,cv=10,n_jobs=8,method='predict_proba')
    predict_save=[Y.astype(int),y_predict.astype(int),y_predict_prob[:,1]]
    predict_save=np.array(predict_save).T
#     pd.DataFrame(predict_save).to_csv(path+name+'_predict.csv',header=None,index=False)
    ROC_AUC_area="%0.4f"%cross_val_score(svm.SVC(kernel='rbf',C=C,gamma=gamma),X,Y,cv=10,n_jobs=-1).mean()
    ACC=metrics.accuracy_score(Y,y_predict)
    precision, recall, SN, SP, GM, TP, TN, FP, FN = performance(Y, y_predict)
    F1_Score=metrics.f1_score(Y, y_predict)
    F_measure=F1_Score
    MCC=metrics.matthews_corrcoef(Y, y_predict)
    pos=TP+FN
    neg=FP+TN
    savedata=[[['svm'+"C:"+str(C)+"gamma:"+str(gamma),ACC,precision, recall,SN, SP, GM,F_measure,F1_Score,MCC,ROC_AUC_area,TP,FN,FP,TN,pos,neg]]]
    if ACC>bestACC:
        bestACC=ACC
        bestgamma=gamma
        bestC=C
    print savedata
    print X.shape[1]
print bestACC
print bestC
print bestgamma
#     easy_excel.save("svm_crossvalidation",[str(X.shape[1])],savedata,path+'cross_validation_'+name+'.xls')


start


/usr/local/lib/python2.7/dist-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [403 496 497 499 500 598 599 600 601 678 679 680 681 758 759 760 761] are constant.
  UserWarning)


[[['svmC:0.25gamma:0.03125', 0.5, 0.5, 1.0, 1.0, 0.0, 0.0, 0.66666666666666663, 0.66666666666666663, 0.0, '0.5000', 1130.0, 0.0, 1130.0, 0.0, 1130.0, 1130.0]]]
1
[[['svmC:0.25gamma:0.03125', 0.5, 0.5, 1.0, 1.0, 0.0, 0.0, 0.66666666666666663, 0.66666666666666663, 0.0, '0.5000', 1130.0, 0.0, 1130.0, 0.0, 1130.0, 1130.0]]]
2
[[['svmC:0.25gamma:0.03125', 0.5, 0.5, 1.0, 1.0, 0.0, 0.0, 0.66666666666666663, 0.66666666666666663, 0.0, '0.5000', 1130.0, 0.0, 1130.0, 0.0, 1130.0, 1130.0]]]
3
[[['svmC:0.25gamma:0.03125', 0.5, 0.5, 1.0, 1.0, 0.0, 0.0, 0.66666666666666663, 0.66666666666666663, 0.0, '0.5000', 1130.0, 0.0, 1130.0, 0.0, 1130.0, 1130.0]]]
4
[[['svmC:0.25gamma:0.03125', 0.5, 0.5, 1.0, 1.0, 0.0, 0.0, 0.66666666666666663, 0.66666666666666663, 0.0, '0.5000', 1130.0, 0.0, 1130.0, 0.0, 1130.0, 1130.0]]]
5
[[['svmC:0.25gamma:0.03125', 0.5, 0.5, 1.0, 1.0, 0.0, 0.0, 0.66666666666666663, 0.66666666666666663, 0.0, '0.5000', 1130.0, 0.0, 1130.0, 0.0, 1130.0, 1130.0]]]
6
[[['svmC:0.25gamma:0.03125',

Process PoolWorker-17008:
Process PoolWorker-17032:
Process PoolWorker-17004:
Process PoolWorker-17010:
Process PoolWorker-17025:
Process PoolWorker-17007:
Process PoolWorker-17006:
Process PoolWorker-16998:
Process PoolWorker-17009:
Process PoolWorker-17029:
Process PoolWorker-17035:
Process PoolWorker-17005:
Process PoolWorker-16995:
Process PoolWorker-16997:
Process PoolWorker-17017:
Process PoolWorker-17038:
Process PoolWorker-17011:
Process PoolWorker-17033:
Process PoolWorker-17002:
Process PoolWorker-17014:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process PoolWorker-17012:
Process PoolWorker-17036:
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process PoolWorker-17034:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, i

KeyboardInterrupt: 

Process PoolWorker-17043:
Traceback (most recent call last):
Process PoolWorker-17044:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most re

In [28]:
"""
    test emsemble classifier
"""
# fertch dataset from pathname
datapath ="data/human_2RFH.csv"
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) / 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) / 2)))
Y.extend(Y2)
y = np.array(Y)

#first split dataset into ten or n subsets, and train GridSearchCV for parameters 
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.1)
svc = svm.SVC()
parameters = {'kernel': ['rbf'], 'C':map(lambda x:2**x,np.linspace(-2,5,7)), 'gamma':map(lambda x:2**x,np.linspace(-5,2,7))}
clf = GridSearchCV(svc, parameters, cv=10, n_jobs=12, scoring='accuracy')
clf.fit(X, y)

# get C and gamma parameters
C=clf.best_params_['C']
gamma=clf.best_params_['gamma']

#define a output collect cans
final_predict=0


for i in xrange(20):
    X_train_sub,_,y_train_sub,_=train_test_split(X_train,y_train,test_size=0.1)
    #define a classifier of svm and predict the results
    svm_classifier=svm.SVC(kernel='rbf',C=C,gamma=gamma)
    svm_classifier.fit(X_train_sub,y_train_sub)
    y_predict=svm_classifier.predict(X_test)

    #get the results of Accuracy
    ACC=metrics.accuracy_score(y_test,y_predict)
    
    print "echo"+str(i+1),"accuracy:"+str(ACC)
    final_predict+=ACC
print "final_accuracy_score:"+str(final_predict/20)

echo1 accuracy:0.889380530973
echo2 accuracy:0.889380530973
echo3 accuracy:0.889380530973
echo4 accuracy:0.889380530973
echo5 accuracy:0.889380530973
echo6 accuracy:0.893805309735
echo7 accuracy:0.889380530973
echo8 accuracy:0.889380530973
echo9 accuracy:0.889380530973
echo10 accuracy:0.889380530973
echo11 accuracy:0.893805309735
echo12 accuracy:0.889380530973
echo13 accuracy:0.889380530973
echo14 accuracy:0.889380530973
echo15 accuracy:0.889380530973
echo16 accuracy:0.889380530973
echo17 accuracy:0.889380530973
echo18 accuracy:0.889380530973
echo19 accuracy:0.889380530973
echo20 accuracy:0.889380530973
final_accuracy_score:0.88982300885


In [39]:
"""
    jackknife xgboost
"""
loop_times=2260
classifier="xgboost"
datapath =path3
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)
parameters = [{'n_estimators':range(1,10,1),
                      'max_depth':range(2,10),
                      'learning_rate':[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
                      'subsample':[0.75,0.8,0.85,0.9]
                      }]
clf = GridSearchCV(XGBClassifier(), parameters, cv=10, n_jobs=1, scoring='accuracy')
clf.fit(X, Y)
n_estimators=clf.best_params_['n_estimators']
max_depth=clf.best_params_['max_depth']
learning_rate=clf.best_params_['learning_rate']
subsample=clf.best_params_['subsample']

final_accuracy_score=0

for i in xrange(loop_times):
    X_train,X_test=np.delete(X,i,0),X[i]
    y_train,y_test=np.delete(Y,i,0),Y[i]
    xgboost_classifier=XGBClassifier(n_estimators=n_estimators,learning_rate=learning_rate,subsample=subsample,max_depth=max_depth)
    xgboost_classifier.fit(X_train,y_train)
    y_predict=xgboost_classifier.predict(X_test)
    ACC=metrics.accuracy_score(y_test,y_predict)
    final_accuracy_score+=ACC
    print "echo"+str(i),"accuracy"+str(ACC)
print "final_accuracy_score"+str(final_accuracy_score/loop_times)

ValueError: Input numpy.ndarray must be 2 dimensional

In [ ]:
"""
    mrmr and svm
"""
import pandas as pd
import numpy as np
loop_times=2260
classifier="svm"
datapath =path3
train_data = pd.read_csv(datapath, header=None, index_col=None)
X = np.array(train_data)
Y = list(map(lambda x: 1, xrange(len(train_data) // 2)))
Y2 = list(map(lambda x: 0, xrange(len(train_data) // 2)))
Y.extend(Y2)
Y = np.array(Y)